In [25]:
include("src/CombinedSubexpressions.jl")

CombinedSubexpressions

In [27]:
@CombinedSubexpressions.cse inv(H) * G + W * (inv(H) + W)

begin 
    ##invH#301 = inv(H)
    ##*##invH#301G#302 = ##invH#301 * G
    ##+##invH#301W#303 = ##invH#301 + W
    ##*W##+##invH#301W#303#304 = W * ##+##invH#301W#303
    ##+##*##invH#301G#302##*W##+##invH#301W#303#304#305 = ##*##invH#301G#302 + ##*W##+##invH#301W#303#304
    ##+##*##invH#301G#302##*W##+##invH#301W#303#304#305
end


LoadError: UndefVarError: H not defined

In [28]:
function g(x)
    println("g")
    1
end

function f(x)
    println("f")
    2
end

function h(x, y)
    println("h")
    3
end

x = 1

expr = quote
    function foo(x)
        [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
    end
end

quote  # In[28], line 19:
    function foo(x) # In[28], line 20:
        [h(h(f(g(x)),g(x)),i) == i for i = 1:5]
    end
end

In [29]:
CombinedSubexpressions.cacheify!(expr)

quote 
    begin  # In[28], line 19:
        function foo(x)
            ##gx#306 = g(x)
            ##f##gx#306#307 = f(##gx#306)
            ##h##f##gx#306#307##gx#306#308 = h(##f##gx#306#307,##gx#306)
            begin  # In[28], line 20:
                [h(##h##f##gx#306#307##gx#306#308,i) == i for i = 1:5]
            end
        end
    end
end

In [30]:
@CombinedSubexpressions.cse function foo(x)
    [f(x) == i for i in 1:5]
end

begin 
    function foo(x)
        ##fx#309 = f(x)
        begin  # In[30], line 2:
            [##fx#309 == i for i = 1:5]
        end
    end
end


foo (generic function with 1 method)

In [31]:
foo(1)

f


5-element Array{Bool,1}:
 false
  true
 false
 false
 false

In [32]:
@CombinedSubexpressions.cse begin
    [h(h(f(g(x)), g(x)), i) == i for i in 1:5]
end

begin 
    ##gx#310 = g(x)
    ##f##gx#310#311 = f(##gx#310)
    ##h##f##gx#310#311##gx#310#312 = h(##f##gx#310#311,##gx#310)
    begin  # In[32], line 2:
        [h(##h##f##gx#310#311##gx#310#312,i) == i for i = 1:5]
    end
end
g
f
h
h
h
h
h
h


5-element Array{Bool,1}:
 false
 false
  true
 false
 false

In [33]:
@CombinedSubexpressions.cse begin
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    ##zeros5#313 = zeros(5)
    ##g5#314 = g(5)
    begin  # In[33], line 2:
        x = ##zeros5#313 # In[33], line 3:
        for i = 1:5 # In[33], line 4:
            x[i] = f(x[i]) + ##g5#314
        end
    end
end
g
f
f
f
f
f


In [34]:
@CombinedSubexpressions.cse function bar(y)
    x = zeros(5)
    for i in 1:5
        x[i] = f(x[i]) + g(5)
    end
end

begin 
    function bar(y)
        ##zeros5#315 = zeros(5)
        ##g5#316 = g(5)
        begin  # In[34], line 2:
            x = ##zeros5#315 # In[34], line 3:
            for i = 1:5 # In[34], line 4:
                x[i] = f(x[i]) + ##g5#316
            end
        end
    end
end


bar (generic function with 1 method)

In [35]:
@code_warntype bar(1)

Variables:
  #self#::#bar
  y::Int64
  zeros5#315::Array{Float64,1}
  g5#316::Int64
  x::Array{Float64,1}
  #temp#::Int64
  i::Int64

Body:
  begin 
      zeros5#315::Array{Float64,1} = $(Expr(:invoke, LambdaInfo for fill!(::Array{Float64,1}, ::Float64), :(Base.fill!), :((Core.ccall)(:jl_alloc_array_1d,(Core.apply_type)(Core.Array,Float64,1)::Type{Array{Float64,1}},(Core.svec)(Core.Any,Core.Int)::SimpleVector,Array{Float64,1},0,5,0)::Array{Float64,1}), :((Base.box)(Float64,(Base.sitofp)(Float64,0)))))
      # meta: location In[28] g 2
      # meta: location coreio.jl println 5
      SSAValue(4) = (Core.typeassert)(Base.STDOUT,Base.IO)::IO
      # meta: pop location
      (Base.print)(SSAValue(4),"g",'\n')::Void
      # meta: pop location
      g5#316::Int64 = 1
      # meta: location In[34]
      x::Array{Float64,1} = zeros5#315::Array{Float64,1} # line 3:
      SSAValue(6) = (Base.select_value)((Base.sle_int)(1,5)::Bool,5,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#::Int